In [ ]:
%tensorflow_version 1.x
%pip install colorama

TensorFlow 1.x selected.


In [ ]:
import numpy as np
import tensorflow as tf
import random
import colorama
from colorama import Fore, Style


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/'Colab Notebooks'/'SentiGAN-persuasion'/'Real_dataset'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks/SentiGAN-persuasion/Real_dataset


In [ ]:
#%cd 'Real_dataset'

In [ ]:
from dataloader import Gen_Data_loader, Dis_Data_loader
import pickle
from generator import Generator
from discriminator import Discriminator

In [ ]:
print(tf.__version__)

1.15.2


In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13473034879889207712, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 10972130577094805283
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2631271241864870437
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11330115994
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6123515821919046431
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [ ]:

#########################################################################################
#  Generator  Hyper-parameters
######################################################################################
EMB_DIM = 200 # embedding dimension
HIDDEN_DIM = 200 # hidden state dimension of lstm cell
MAX_SEQ_LENGTH = 17  # max sequence length
BATCH_SIZE = 64


#########################################################################################
#  Discriminator  Hyper-parameters
#########################################################################################
dis_embedding_dim = 64
dis_filter_sizes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15]
dis_num_filters = [100, 200, 200, 200, 200, 100, 100, 100, 100, 100, 160]
dis_dropout_keep_prob = 0.75
dis_l2_reg_lambda = 0.2
dis_batch_size = 64


In [ ]:
#########################################################################################
#  Basic Training Parameters
#########################################################################################
TOTAL_BATCH = 2000
dataset_path = "../../data/movie/"
emb_dict_file = dataset_path + "imdb_word.vocab"

# imdb corpus
imdb_file_txt = dataset_path + "imdb/imdb_sentences.txt"
imdb_file_id = dataset_path + "imdb/imdb_sentences.id"

# sstb corpus
sst_pos_file_txt = dataset_path + 'sstb/sst_pos_sentences.txt'
sst_pos_file_id = dataset_path + 'sstb/sst_pos_sentences.id'
sst_neg_file_txt = dataset_path + 'sstb/sst_neg_sentences.txt'
sst_neg_file_id = dataset_path + 'sstb/sst_neg_sentences.id'


eval_file = 'save/eval_file.txt'
eval_text_file = 'save/eval_text_file.txt'
negative_file = 'save/generator_sample.txt'
infer_file = 'save/infer/'


In [ ]:
#########################################################################################
#  #####################################################################################  Microtex Persuasion corpus Training Parameters##########################
#########################################################################################
TOTAL_BATCH = 500 #2000
dataset_path = "../persuasion_corpus_v1.0/"
emb_dict_file = dataset_path + "micro_word.vocab"

# sin persuacion corpus
imdb_file_txt = dataset_path + "0.txt"
imdb_file_id = dataset_path + "0.id"
# imdb_file_txt = dataset_path + "imdb/imdb_sentences.txt"
# imdb_file_id = dataset_path + "imdb/imdb_sentences.id"

# 1 y 3 corpus
##sst_pos_file_txt = dataset_path + '3.txt'
##sst_pos_file_id = dataset_path + '3.id'
# sst_pos_file_txt = dataset_path + '5.txt'
# sst_pos_file_id = dataset_path + '5.id'
sst_pos_file_txt = dataset_path + '4.txt'
sst_pos_file_id = dataset_path + '4.id'
sst_neg_file_txt = dataset_path + '1.txt'
sst_neg_file_id = dataset_path + '1.id'


eval_file = 'save_Persuasion/eval_file_Persuasion.txt'
eval_text_file = 'save_Persuasion/eval_text_file_Persuasion.txt'
negative_file = 'save_Persuasion/generator_sample_Persuasion.txt'
infer_file = 'save_Persuasion/infer_Persuasion/'
out_generated_samples ='save_Persuasion/generated_samples_list.txt'


In [ ]:
def generate_samples(sess, trainable_model, generated_num, output_file, vocab_list, if_log=False, epoch=0):
    # Generate Samples
    generated_samples = []
    for _ in range(int(generated_num)):
        generated_samples.extend(trainable_model.generate(sess))

    if if_log:
        mode = 'a'
        if epoch == 0:
            mode = 'w'
        with open(eval_text_file, mode) as fout:
            # id_str = 'epoch:%d ' % epoch
            for poem in generated_samples:
                poem = list(poem)
                if 2 in poem:
                    poem = poem[:poem.index(2)]
                buffer = ' '.join([vocab_list[x] for x in poem]) + '\n'
                fout.write(buffer)

    with open(output_file, 'w') as fout:
        for poem in generated_samples:
            poem = list(poem)
            if 2 in poem:
                poem = poem[:poem.index(2)]
            buffer = ' '.join([str(x) for x in poem]) + '\n'
            fout.write(buffer)


def generate_infer(sess, trainable_model, epoch, vocab_list):
    generated_samples = []
    for _ in range(int(100)):
        # generated_samples.extend(trainable_model.infer(sess))
        generated_samples.extend(trainable_model.generate(sess))
    file = infer_file+str(epoch)+'.txt'
    with open(file, 'w') as fout:
        for poem in generated_samples:
            poem = list(poem)
            if 2 in poem:
                poem = poem[:poem.index(2)]
            buffer = ' '.join([vocab_list[x] for x in poem]) + '\n'
            fout.write(buffer)
    print("%s saves" % file)
    return


def produce_samples(generated_samples):
    produces_sample = []
    for poem in generated_samples:
        poem_list = []
        for ii in poem:
            if ii == 0:  # _PAD
                continue
            if ii == 2:  # _EOS
                break
            poem_list.append(ii)
        produces_sample.append(poem_list)
    return produces_sample


def load_emb_data(emb_dict_file):
    word_dict = {}
    word_list = []
    item = 0
    with open(emb_dict_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            word = line.strip()
            word_dict[word] = item
            item += 1
            word_list.append(word)
    length = len(word_dict)
    print("Load embedding success! Num: %d" % length)
    return word_dict, length, word_list


def pre_train_epoch(sess, trainable_model, data_loader):
    # Pre-train the generator using MLE for one epoch
    supervised_g_losses = []
    data_loader.reset_pointer()

    for it in range(200):  # data_loader.num_batch):
        batch = data_loader.next_batch()
        _, g_loss = trainable_model.pretrain_step(sess, batch)
        supervised_g_losses.append(g_loss)

    return np.mean(supervised_g_losses)


In [ ]:
vocab_dict, vocab_size, vocab_list = load_emb_data(emb_dict_file)
print(Fore.BLUE + "EMB DATA LOADED")
print(vocab_dict)

print(Fore.GREEN + "Creating batches")
# prepare data
pre_train_data_loader = Gen_Data_loader(BATCH_SIZE, vocab_dict)
pre_train_data_loader.create_batches([imdb_file_id, sst_pos_file_id, sst_neg_file_id])

gen_data_loader = Gen_Data_loader(BATCH_SIZE, vocab_dict)

gen_data_loader.create_batches([sst_pos_file_id, sst_neg_file_id])

dis_data_loader = Dis_Data_loader(BATCH_SIZE, vocab_dict, MAX_SEQ_LENGTH)

from tensorflow.python.framework import ops
ops.reset_default_graph()

#### build model
#### num_emb, vocab_dict, batch_size, emb_dim, num_units, sequence_length


Load embedding success! Num: 4723
EMB DATA LOADED
{'<PAD>': 0, '<UNK>': 1, '<EOS>': 2, '<GO>': 3, 'you': 4, 'to': 5, 'the': 6, 'and': 7, 'i': 8, 'that': 9, 'a': 10, 'of': 11, 'know': 12, 'in': 13, 'it': 14, 'we': 15, 'is': 16, 'me': 17, 'on': 18, 'they': 19, 'im': 20, 'what': 21, 'get': 22, '<name>': 23, 'have': 24, 'out': 25, 'this': 26, 'all': 27, 'dont': 28, 'be': 29, 'with': 30, 'but': 31, 'not': 32, 'do': 33, 'going': 34, 'there': 35, 'if': 36, 'for': 37, 'just': 38, 'can': 39, 'here': 40, 'gonna': 41, 'your': 42, 'are': 43, 'want': 44, 'he': 45, 'so': 46, 'now': 47, 'my': 48, 'up': 49, 'got': 50, 'like': 51, 'okay': 52, 'was': 53, 'thats': 54, 'its': 55, 'them': 56, 'her': 57, 'as': 58, 'were': 59, 'uh': 60, 'about': 61, 'right': 62, 'go': 63, 'well': 64, 'come': 65, 'or': 66, 'see': 67, 'when': 68, 'no': 69, 'because': 70, 'at': 71, 'talk': 72, 'said': 73, 'one': 74, 'she': 75, 'then': 76, 'youre': 77, 'down': 78, 'him': 79, 'people': 80, 'tell': 81, 'let': 82, 'will': 83, 'they

In [ ]:
generator = Generator(vocab_size, vocab_dict, BATCH_SIZE, EMB_DIM, HIDDEN_DIM, MAX_SEQ_LENGTH)




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.cast` instead.

Instructions for updating:
Use `tf.cast` instead.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
discriminator = Discriminator(sequence_length=MAX_SEQ_LENGTH, num_classes=2,
                              vocab_size=vocab_size,
                              embedding_size=dis_embedding_dim,
                              filter_sizes=dis_filter_sizes, num_filters=dis_num_filters,
                              l2_reg_lambda=dis_l2_reg_lambda)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.




In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

####### stop here if you wanna load a trained model #############\

In [ ]:
log = open('save/experiment-log.txt', 'w')

buffer = 'Start pre-training generator...'
print(buffer)
log.write(buffer + '\n')
for epoch in range(120):  #120 #150
    train_loss = pre_train_epoch(sess, generator, pre_train_data_loader)
    if epoch % 5 == 0:
        generate_samples(sess, generator, 1, eval_file, vocab_list, if_log=True, epoch=epoch)
        print('    pre-train epoch ', epoch, 'train_loss ', train_loss)
        buffer = '    epoch:\t' + str(epoch) + '\tnll:\t' + str(train_loss) + '\n'
        log.write(buffer)

Start pre-training generator...
    pre-train epoch  0 train_loss  6.1339016
    pre-train epoch  5 train_loss  3.8397105
    pre-train epoch  10 train_loss  2.6367326
    pre-train epoch  15 train_loss  1.9698237
    pre-train epoch  20 train_loss  1.6334823
    pre-train epoch  25 train_loss  1.4907056
    pre-train epoch  30 train_loss  1.3675961
    pre-train epoch  35 train_loss  1.3861121
    pre-train epoch  40 train_loss  1.2238063
    pre-train epoch  45 train_loss  1.1670192
    pre-train epoch  50 train_loss  1.5814325
    pre-train epoch  55 train_loss  1.1207546
    pre-train epoch  60 train_loss  1.0423975
    pre-train epoch  65 train_loss  2.0141416
    pre-train epoch  70 train_loss  1.2233944
    pre-train epoch  75 train_loss  1.0522007
    pre-train epoch  80 train_loss  1.1298189
    pre-train epoch  85 train_loss  1.4363676
    pre-train epoch  90 train_loss  1.0950615
    pre-train epoch  95 train_loss  1.0472655
    pre-train epoch  100 train_loss  1.8218226
   

In [ ]:
buffer = 'Start pre-training discriminator...'
print(buffer)
#log.write(buffer)
################## para borrar 
dis_data_loader = Dis_Data_loader(BATCH_SIZE, vocab_dict, MAX_SEQ_LENGTH)

for _ in range(10):   # 10
    generate_samples(sess, generator, 70, negative_file, vocab_list)
    print("discriminador")
    dis_data_loader.load_train_data([sst_pos_file_id, sst_neg_file_id], [negative_file])
    for _ in range(3):
        dis_data_loader.reset_pointer()
        for it in range(dis_data_loader.num_batch):
            x_batch, y_batch = dis_data_loader.next_batch()
            feed = {
                discriminator.input_x: x_batch,
                discriminator.input_y: y_batch,
                discriminator.dropout_keep_prob: dis_dropout_keep_prob,
            }
            d_loss, d_acc, _ = sess.run([discriminator.loss, discriminator.accuracy, discriminator.train_op], feed)
    buffer = "discriminator loss %f acc %f" % (d_loss, d_acc)
    print(buffer)
    log.write(buffer + '\n')

Start pre-training discriminator...
discriminador
(24084, 17)
discriminator loss 1.463332 acc 0.906250
discriminador
(24084, 17)
discriminator loss 0.652759 acc 1.000000
discriminador
(24084, 17)
discriminator loss 0.309706 acc 0.984375
discriminador
(24084, 17)
discriminator loss 0.120282 acc 0.968750
discriminador
(24084, 17)
discriminator loss 0.039636 acc 1.000000
discriminador
(24084, 17)
discriminator loss 0.023978 acc 1.000000
discriminador
(24084, 17)
discriminator loss 0.017007 acc 1.000000
discriminador
(24084, 17)
discriminator loss 0.004271 acc 1.000000
discriminador
(24084, 17)
discriminator loss 0.026238 acc 0.984375
discriminador
(24084, 17)
discriminator loss 0.002318 acc 1.000000


In [ ]:
def build_from_ids(vv, vocab_list):
    a = []
    for i in vv:
        a.append(vocab_list[i])
    return(' '.join(a))


In [ ]:
print("Start Adversarial Training...")
log.write('adversarial training...')
for total_batch in range(TOTAL_BATCH):
    # Train the generator
    for it in range(2):
        # print("1")
        samples = generator.generate(sess)
        samples = produce_samples(samples)
        # print("2")
        rewards = generator.get_reward(sess, samples, 16, discriminator)
        # print("3")
        a = str(samples[0])
        b = str(rewards[0])
        # rewards = change_rewards(rewards)
        # c = str(rewards[0])
        d = build_from_ids(samples[0], vocab_list)
        buffer = "%s\n%s\n%s\n\n" % (d, a, b)
        print(buffer)
        log.write(buffer)

        # print("4")
        rewards_loss = generator.update_with_rewards(sess, samples, rewards)
        # print("5")
        # good rewards
        # good_samples = gen_data_loader.next_batch()
        # rewards = np.array([[0.0001] * SEQ_LENGTH] * BATCH_SIZE)
        # a = str(good_samples[0])
        # b = str(rewards[0])
        # buffer = "%s\n%s\n\n" % (a, b)
        # print(buffer)
        # log.write(buffer)
        # rewards_loss = generator.update_with_rewards(sess, good_samples, rewards, START_TOKEN)

        # little1 good reward
        little1_samples = gen_data_loader.next_batch()
        rewards = generator.get_reward(sess, little1_samples, 16, discriminator)
        a = str(little1_samples[0])
        b = str(rewards[0])
        buffer = "%s\n%s\n\n" % (a, b)
        # print(buffer)
        log.write(buffer)
        rewards_loss = generator.update_with_rewards(sess, little1_samples, rewards)

    # generate_infer(sess, generator, epoch, vocab_list)

    # Test
    if total_batch % 5 == 0 or total_batch == TOTAL_BATCH - 1:
        generate_samples(sess, generator, 120, eval_file, vocab_list, if_log=True)
        generate_infer(sess, generator, total_batch, vocab_list)
        # generate_samples(sess, generator, BATCH_SIZE, generated_num, eval_file)
        # likelihood_data_loader.create_batches(eval_file)
        # test_loss = target_loss(sess, target_lstm, likelihood_data_loader)
        buffer = 'reward-train epoch %s train loss %s' % (str(total_batch), str(rewards_loss))
        print(buffer)
        log.write(buffer + '\n')
        generator.save_model(sess)

    # Train the discriminator
    begin = True
    for _ in range(1):
        generate_samples(sess, generator, 70, negative_file, vocab_list)
        dis_data_loader.load_train_data([sst_pos_file_id, sst_neg_file_id], [negative_file])
        for _ in range(3):
            dis_data_loader.reset_pointer()
            for it in range(dis_data_loader.num_batch):
                x_batch, y_batch = dis_data_loader.next_batch()
                feed = {
                    discriminator.input_x: x_batch,
                    discriminator.input_y: y_batch,
                    discriminator.dropout_keep_prob: dis_dropout_keep_prob,
                }
                d_loss, d_acc, _ = sess.run([discriminator.loss, discriminator.accuracy, discriminator.train_op],
                                            feed)
                if (total_batch % 5 == 0 or total_batch == TOTAL_BATCH - 1) and begin:
                    buffer = "discriminator loss %f acc %f\n" % (d_loss, d_acc)
                    print(buffer)
                    log.write(buffer)
                    begin = False

    # pretrain
    for _ in range(10):
        train_loss = pre_train_epoch(sess, generator, pre_train_data_loader)


Start Adversarial Training...
okay he can go youve got mine i mean so we can talk alot better now im
[52, 45, 39, 63, 213, 50, 880, 8, 113, 46, 15, 39, 72, 1548, 238, 47, 20]
[0.9774264  0.93415725 0.9995664  0.9999491  0.9999627  0.9999521
 0.9999965  0.9999913  0.9990626  0.99821985 0.99324137 0.99613
 0.99998    0.99998766 0.99998766 0.9999484  0.99998766]


and some of its with a walk off the plane lets face it you coulda slinged out
[7, 86, 11, 55, 30, 10, 196, 145, 6, 217, 195, 570, 14, 4, 1741, 2873, 25]
[0.94465363 0.99984664 0.9499255  0.9830577  0.9992769  0.9728359
 0.98757297 0.99981225 0.9985358  0.99958205 0.99997145 0.99997145
 0.9985426  0.99997145 0.99997145 0.9999775  0.99997145]


save_Persuasion/infer_Persuasion/0.txt saves
reward-train epoch 0 train loss 5.1809263
save model model4_roller_windows success!
(24084, 17)
discriminator loss 0.003458 acc 1.000000

and ye perish in the way when is wrath is kindled but a little now im between
[7, 859, 2798, 13, 6, 99, 68, 

In [ ]:
#################### Start de loaded model here

In [ ]:
generator.load_model(sess)
# generator.load_this_model(sess,'save_Persuasion/ckpt/model5_roller_windows.ckpt')
samples = []
samples = generator.generate(sess)
samples = produce_samples(samples)

a = str(samples[0])
d = build_from_ids(samples[0], vocab_list)
buffer = "%s\n%s\n\n" % (d, a)
print(buffer)
        

INFO:tensorflow:Restoring parameters from save_Persuasion/ckpt/model4_roller_windows.ckpt
save model model4_roller_windows success!
we want to battle what the hell of a few good jim getting <UNK> her you know
[15, 44, 5, 1417, 21, 6, 262, 11, 10, 349, 120, 356, 188, 1, 57, 4, 12]




In [ ]:
 sentences = []
for i in range(500):
  samples = []
  samples = generator.generate(sess)
  samples = produce_samples(samples)


  a = str(samples[0])
  d = build_from_ids(samples[0], vocab_list)
  sentences.append(d)
  buffer = "%s\n%s\n\n" % (d, a)
  # print(buffer)
# print(sentences)
out_generated_samples ='save_Persuasion/generated_samples_list_model4.txt'

with open(out_generated_samples, 'w') as filehandle:
    for listitem in sentences:
        filehandle.write('%s\n' % listitem)
